# espagne.csv + france.csv + maroc.csv + petite.csv

# Data parsing

In [1]:
using PyPlot: pygui
pygui(true)
using Plots
pyplot()

Plots.PyPlotBackend()

In [2]:
include("import_all.jl")

write_solution_to_file (generic function with 1 method)

## Reading the instance

In [3]:
function read_instance(instance_file)
    data = open(joinpath("..", "sujet", instance_file)) do file
        readlines(file)
    end

    dims = lire_dimensions(data[1])

    emballages = [lire_emballage(data[1+e], dims) for e = 1:dims.E]

    usines = [lire_usine(data[1+dims.E+u], dims) for u = 1:dims.U]

    fournisseurs = [
        lire_fournisseur(data[1+dims.E+dims.U+f], dims) for f = 1:dims.F
    ]

    graphe = lire_graphe(data[1+dims.E+dims.U+dims.F+1:end], dims)

    instance = lire_instance(joinpath("..", "sujet", instance_file))

#     plot_sites(instance)
    return dims, emballages, usines,fournisseurs,graphe, instance
end

read_instance (generic function with 1 method)

# Solution

In [4]:
using JuMP
using Gurobi
import LightGraphs
const lg = LightGraphs

LightGraphs

## Func def

In [5]:
function create_graph(G_original, dims, us, fs, es, J_init, J_fin, e)
    U = dims.U
    F = dims.F
    J = J_fin - J_init + 1
    # J = dims.J
    cost_dispatch = dims.γ
    L = dims.L
    
    G = SimpleDiGraph((3 * U + 4 * F) * J + U + F + 1)
    capacity = zeros((3 * U + 4 * F) * J + U + F + 1, 
        (3 * U + 4 * F) * J + U + F + 1)
    cost_mat = zeros((3 * U + 4 * F) * J + U + F + 1, 
        (3 * U + 4 * F) * J + U + F + 1)
    
    for j = 1:J # for each day
        for u = 1:U # connect each u to each f stock
            for f = 1:F
                add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, 
                    (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1)
                capacity[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, 
                    (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1] += Inf
                cost_mat[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, 
                    (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1] += (G_original.d[u, U + f] * cost_dispatch + dims.ccam + dims.cstop) / L * es[e].l
            end
        end
        
        for f = 1:F
            # connect fictive u (carton) to each f consommation
            add_edge!(G, nv(G), (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * f)
            #add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * f, nv(G)) # cycle
            capacity[nv(G), (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * f] += Inf
            #capacity[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * f, nv(G)] += Inf
            cost_mat[nv(G), (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * f] += fs[f].cexc[e]
            # cost_mat[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * f, nv(G)] += fs[f].cexc[e]
        end
    end
    
    for j = 1:J
        for u = 1:U
            # connect u day j to two additional vertices
            add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 2)
            add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 3)
            
            capacity[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 2] += us[u].r[e, j + J_init - 1]
            capacity[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 3] += Inf
            
            cost_mat[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 3] += us[u].cs[e]
            
            if j < J
                # connect two additional vertices to u day j+1
                add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 2, 
                    j * (3 * U + 4 * F) + 3 * (u - 1) + 1)
                add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 3, 
                    j * (3 * U + 4 * F) + 3 * (u - 1) + 1)
                
                capacity[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 2, 
                    j * (3 * U + 4 * F) + 3 * (u - 1) + 1] += us[u].r[e, j + J_init - 1]
                capacity[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 3, 
                    j * (3 * U + 4 * F) + 3 * (u - 1) + 1] += Inf
            end
        end
        
        for f = 1:F
            # connect f stock day j to two additional vertices
            add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 2)
            add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 3)
            
            capacity[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 2] += fs[f].r[e, j + J_init - 1]
            capacity[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 3] += Inf
            
            cost_mat[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1, 
                (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 3] += fs[f].cs[e]
            
            if j < J
                # connect each f stock to f consommation of the next day
                add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1, j * (3 * U + 4 * F) + 3 * U + 4 * f)
                capacity[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1, j * (3 * U + 4 * F) + 3 * U + 4 * f] += Inf
                
                # connect two additional vertices to f day j+1
                add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 2, 
                    j * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1)
                add_edge!(G, (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 3, 
                    j * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1)
                
                capacity[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 2, 
                    j * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1] += fs[f].r[e, j + J_init - 1]
                capacity[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 3, 
                    j * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1] += Inf
            end
        end
    end
    
    # connect initial stock to the stock of day 1 (for each u)
    for u = 1:U
        add_edge!(G, (3 * U + 4 * F) * J + u, 3 * (u - 1) + 1)
        
        capacity[(3 * U + 4 * F) * J + u, 3 * (u - 1) + 1] += Inf
    end
    
    # connect initial stock to the stock of day 1 (for each f) & to the f consommation of day 1
    for f = 1:F
        add_edge!(G, (3 * U + 4 * F) * J + U + f, 3 * U + 4 * (f - 1) + 1)  
        add_edge!(G, (3 * U + 4 * F) * J + U + f, 3 * U + 4 * f)
        
        capacity[(3 * U + 4 * F) * J + U + f, 3 * U + 4 * (f - 1) + 1] += Inf
        capacity[(3 * U + 4 * F) * J + U + f, 3 * U + 4 * f] += Inf
    end
    
    return G, capacity, cost_mat
end

create_graph (generic function with 1 method)

In [6]:
# for a given e
function set_demand(dims, us, fs, J_init, J_fin, e) # b parameter
    U = dims.U
    F = dims.F
    J = J_fin - J_init + 1
    # J = dims.J
    demand = zeros((3 * U + 4 * F) * J + U + F + 1)
    
    for j = 1:J
        # for each u
        for u = 1:U
            demand[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1] -= us[u].b⁺[e, j + J_init - 1]
        end
        
        # for each f consommation
        for f = 1:F
            demand[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * f] += fs[f].b⁻[e, j + J_init - 1]
        end
    end
    
    # for initial stock
    if J_init <= 1
        for u = 1:U
            demand[J * (3 * U + 4 * F) + u] -= us[u].s0[e]
        end
    
        for f = 1:F
            demand[J * (3 * U + 4 * F) + U + f] -= fs[f].s0[e]
        end
    else
        for u = 1:U
            demand[J * (3 * U + 4 * F) + u] -= us[u].s[e, J_init - 1]
        end
        
        for f = 1:F
            demand[J * (3 * U + 4 * F) + U + f] -= fs[f].s[e, J_init - 1]
        end
    end
    
    println(sum(demand))
    # additional vertices from day 1 to day J-1: b = 0
    # additional vertices of day J: b >= 0
    demand[(3 * U + 4 * F) * J + U + F + 1] += Inf # +Inf means that b is not fixed
    for u = 1:U
        demand[(J - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 2] -= Inf # -Inf means b>=0
        demand[(J - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 3] -= Inf
    end
    for f = 1:F
        demand[(J - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 2] -= Inf
        demand[(J - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 3] -= Inf
    end
    
    return demand
end

set_demand (generic function with 1 method)

In [7]:
function min_cost_flow(g, node_demand, edge_capacity, edge_cost, optimizer)
    m = JuMP.Model(optimizer)
    vtxs = vertices(g)
    
    source_nodes = [v for v in vtxs if node_demand[v] < 0 && node_demand[v] != -Inf]
    sink_nodes = [v for v in vtxs if node_demand[v] > 0 && node_demand[v] != Inf]
    
    idx_dict = Dict()
    ridx_dict = Dict()
    i = 1
    for e in lg.edges(g)
        idx_dict[i] = [src(e), dst(e)]
        ridx_dict[(src(e), dst(e))] = i
        i += 1
    end
    
    @variable(m, 0 <= f[i = 1:ne(g)] <= edge_capacity[idx_dict[i][1], idx_dict[i][2]], Int)
    @objective(m, Min, sum(f[i] * edge_cost[idx_dict[i][1], idx_dict[i][2]] for i = 1:ne(g)))
    # @variable(m, 0 <= f[i=vtxs, j=vtxs; (i,j) in lg.edges(g)] <= edge_capacity[i, j])
    # @objective(m, Min, sum(f[src(e),dst(e)] * edge_cost[src(e), dst(e)] for e in lg.edges(g)))

    for v in lg.vertices(g)
        if v in source_nodes
            @constraint(m,
                sum(f[ridx_dict[(v, vout)]] for vout in outneighbors(g, v)) - sum(f[ridx_dict[(vin, v)]] for vin in lg.inneighbors(g, v)) == -node_demand[v]
            )
        elseif v in sink_nodes
            @constraint(m,
                sum(f[ridx_dict[(vin, v)]] for vin in lg.inneighbors(g, v)) - sum(f[ridx_dict[(v, vout)]] for vout in outneighbors(g, v)) == node_demand[v]
            )
        else
            if node_demand[v] == -Inf
                @constraint(m, sum(f[ridx_dict[(vin, v)]] for vin in lg.inneighbors(g, v)) - sum(f[ridx_dict[(v, vout)]] for vout in outneighbors(g, v)) >= 0)
            elseif node_demand[v] != Inf
                @constraint(m,
                    sum(f[ridx_dict[(vin, v)]] for vin in lg.inneighbors(g, v)) == sum(f[ridx_dict[(v, vout)]] for vout in outneighbors(g, v))
                )
            end
        end
    end

    optimize!(m)
    ts = termination_status(m)
    result_flow = spzeros(nv(g), nv(g))
    if ts != MOI.OPTIMAL
        @warn "Problem does not have an optimal solution, status: $(ts)"
        return result_flow
    end
    for e in lg.edges(g)
        (i,j) = Tuple(e)
        result_flow[i,j] = JuMP.value(f[ridx_dict[(i,j)]])
    end
    return result_flow
    
end

min_cost_flow (generic function with 1 method)

In [8]:
function run_opt(g, dims, us, fs, es, J_init, J_fin, e, optimizer)
    g, capacity, cost_mat = create_graph(g, dims, us, fs, es, J_init, J_fin, e)
    demand = set_demand(dims, us, fs, J_init, J_fin, e)
    flow = min_cost_flow(g, demand, capacity, cost_mat, optimizer)
    return flow
end

run_opt (generic function with 1 method)

In [9]:
function read_flow(flow, U, F, J_init, J_fin)
    # create a U * F * J matrix (for a given e) for dispatching
    # create a U * J matrix (for a given e) for stockage
    # create a F * J matrix (for a given e) for stockage
    # create a F * J matrix (for a given e) for carton consommation
    J = J_fin - J_init + 1
    dispatch = zeros(U, F, J)
    stock_U = zeros(U, J)
    stock_F = zeros(F, J)
    consom_carton = zeros(F, J)
    for j = 1:J
        for u = 1:U
            for f = 1:F
                dispatch[u, f, j] += flow[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1,
                    (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1]
            end
        end
        
        for u = 1:U
            stock_U[u, j] = flow[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 2] +
            flow[(j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 1, (j - 1) * (3 * U + 4 * F) + 3 * (u - 1) + 3]
        end
        
        for f = 1:F
            stock_F[f, j] = flow[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1,
                (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 2] + 
            flow[(j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 1,
                (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * (f - 1) + 3]
            
            consom_carton[f, j] = flow[size(flow)[1], (j - 1) * (3 * U + 4 * F) + 3 * U + 4 * f]
        end
    end
    
    return dispatch, consom_carton, stock_U, stock_F
end

read_flow (generic function with 1 method)

# First Fit Decreasing

In [10]:
function get_disp_all_E(graphe, dims, usines, fournisseurs, emballages)
    disp_all_E = zeros(dims.U,dims.F,dims.J,dims.E)
    for e in 1:dims.E
        flow = run_opt(graphe, dims, usines, fournisseurs, emballages, 1, dims.J, e, Gurobi.Optimizer)
        disp, consom_carton, stock_U, stock_F = read_flow(flow, dims.U, dims.F, 1, dims.J)
        disp_all_E[:,:,:,e] = disp[:,:,:]
    end
        return disp_all_E
end


function get_F_list(u,j,dims,disp_all_E)
    FEflow_uj = Dict{Int64,Array{Int64}}()
    F_totalE = Dict{Int64, Int64}()
    F_list = []
    disp_all_E
    for f in 1:dims.F
        if sum(disp_all_E[u,f,j,:])>0
            FEflow_uj[f] = disp_all_E[u,f,j,:]
            F_totalE[f] = sum(disp_all_E[u,f,j,:])
            append!(F_list, f)
        end
    end
    return F_list
end

function get_items(u,f,j,disp_all_E,le,E)
    qe = disp_all_E[u,f,j,:]
    items = []
    for i in 1:E
        if qe[i] != 0
            for j in 1:qe[i]
                push!(items,le[i,:])
            end
        end
    end
    return items
end


function heur_FFD(L, items)
    n = length(items)
    order = sortperm(items,by=it->it[2], rev=true)
    bin_for_items = fill(-1,n)
    bin_space = []
    for i in order
        if length(bin_space) > 0
            for j in 1:length(bin_space)
                if items[i][2] <= bin_space[j]
                    bin_for_items[i] = j
                    bin_space[j] -= items[i][2]
                    break
                end
            end
        end

        if bin_for_items[i] < 0
            j = length(bin_space)
            bin_for_items[i] = j+1
            append!(bin_space, L-items[i][2])
        end
    end
    n_bins = length(bin_space)
    items_for_bins = []
    for n in 1:n_bins
        n_items = []
        for it in order
            if bin_for_items[it] == n
                push!(n_items, items[it])
            end
        end
        push!(items_for_bins,n_items)
    end
    return n_bins,bin_space,items_for_bins
end

function count_items(items)
    items_count = Dict{Int64, Int64}()
    for it in items
        if it[1] in collect(keys(items_count))
            items_count[it[1]] += 1
        else
            items_count[it[1]] = 1
        end
    end
    return collect(items_count)
end

function count_items_list(items)
    items_count = Dict{Int64, Int64}()
    for i in 1:length(items)
        if items[i][1] in collect(keys(items_count))
            items_count[items[i][1]] += 1
        else
            items_count[items[i][1]] = 1
        end
    end
    return items_count
end

# calculate first big f

function get_route_1f(camion_space, camion_stop, u,j,F_list, dims,disp_all_E, le)
    rest_items = Dict{Int64,Array}()
    fini_F_list = []
    for f in F_list
        items = get_items(u,f,j,disp_all_E,le,dims.E)
        n_bins,bin_space,items_for_bins = heur_FFD(dims.L, items)
        for n in 1:n_bins
            if bin_space[n] > dims.L * 0.95
                if f in collect(keys(rest_items))
                    append!(rest_items[f],items_for_bins[n])
                else
                    rest_items[f] = items_for_bins[n]
                end
            else
#                 sol_line = write_sol_route_1f(r, u, j, f, items_for_bins[n])
#                 r += 1
#                 push!(sol_string,sol_line)
                append!(camion_space, bin_space[n])
                push!(camion_stop, [f=>[items_for_bins[n]],])
            end
        end
    end
    return rest_items,camion_space, camion_stop
end

function f_heur_FFD(L, items, f, camion_space, camion_stop)
    n = length(items)
    order = sortperm(items,by=it->it[2], rev=true)
    bin_for_items = fill(-1,n)
    for i in order
        if length(camion_space) > 0
            for j in 1:length(camion_space)
                if length(camion_stop[j]) < 4 && items[i][2] <= camion_space[j]
                    for st in length(camion_stop[j])
                        if camion_stop[j][st][1] == f
                            push!(camion_stop[j][st][2], items[i])
                        else
                            append!(camion_stop[j], [f=>[items[i],],])
                        end
                    end
                    bin_for_items[i] = j
                    camion_space[j] -= items[i][2]
                    break 
                end
            end
        end

        if bin_for_items[i] < 0
            j = length(camion_space)
            push!(camion_stop, [f=>[items[i],],])
            append!(camion_space, L-items[i][2])
        end
    end
    return camion_space, camion_stop
end

# routing for rest f

function get_route_multi_f(u,j,rest_items,camion_space,camion_stop,dims,usines,fournisseurs,graphe)
    point_depart = usines[u].v
    rest_F_open = collect(keys(rest_items))
    while length(rest_F_open)>0
        rest_F_open = sort(rest_F_open, by=f->graphe.d[point_depart, fournisseurs[f].v])
        f = rest_F_open[1]
        items = rest_items[f]
        camion_space, camion_stop = f_heur_FFD(dims.L, items, f, camion_space, camion_stop)
        point_depart = fournisseurs[f].v
        popfirst!(rest_F_open)
    end
    return camion_space, camion_stop
end

function write_sol_route_multi_f(r, u, j, camion_stop, sol_string)
    sol_line = ""
    for c in 1:length(camion_stop) # for each camion
        sol_line = "r " * string(r-1) * " j "* string(j-1) * " x " * string(1) * " u " * string(u-1)
        sol_line = sol_line * " F " * string(length(camion_stop[c]))
        for i in 1:length(camion_stop[c]) # for each stop of camion
            f = camion_stop[c][i][1] 
            sol_line = sol_line * " f " * string(f-1)
            items_nb = count_items_list(camion_stop[c][i][2][1]) # count items of f in camion

            for e in 1:dims.E
                if e in collect(keys(items_nb))
                    q_e = items_nb[e]
                else
                    q_e = 0
                end
                sol_line = sol_line * " e " * string(e-1) * " q " * string(q_e)
            end

        end
        push!(sol_string,sol_line)
        r += 1
        sol_line = ""
    end
    return sol_string, r
end

function run_heur_route(disp_all_E, graphe, dims, usines, fournisseurs, emballages)
    le = Matrix(undef, dims.E, 2)
    for e in 1: dims.E
        le[e,1] = emballages[e].e
        le[e, 2] = emballages[e].l
    end
    sol_string = Array{String,1}(undef,1)
    r = 1
    for u in 1:dims.U
        for j in 1:dims.J
            camion_space = []
            camion_stop = []
            F_list = get_F_list(u,j,dims,disp_all_E)
            rest_items,camion_space, camion_stop = get_route_1f(camion_space, camion_stop, u,j,F_list, dims, disp_all_E, le)
            camion_space, camion_stop = get_route_multi_f(u,j,rest_items,camion_space,camion_stop,dims,usines,fournisseurs,graphe)
            if length(camion_stop)>0
                sol_string, r = write_sol_route_multi_f(r, u, j, camion_stop, sol_string)
            end
        end
    end
    
    sol_string[1] = "R " * string(length(sol_string)-1)
    
    return sol_string
end

run_heur_route (generic function with 1 method)

In [19]:
instance_file = "france.csv"
dims, emballages, usines,fournisseurs,graphe, instance = read_instance(instance_file)
disp_all_E = get_disp_all_E(graphe, dims, usines, fournisseurs, emballages)

-287.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16536 rows, 78258 columns and 152652 nonzeros
Model fingerprint: 0x0e964412
Variable types: 0 continuous, 78258 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 7e+01]
  RHS range        [1e+00, 1e+01]
Presolve removed 12525 rows and 43056 columns
Presolve time: 0.12s
Presolved: 4011 rows, 35202 columns, 69882 nonzeros
Variable types: 0 continuous, 35202 integer (3727 binary)

Root relaxation: objective 7.929231e+03, 37265 iterations, 4.23 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7929.2307692 7929.23077  0.00%     -    4s

Exp

Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16536 rows, 78258 columns and 152652 nonzeros
Model fingerprint: 0x925300a0
Variable types: 0 continuous, 78258 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 5e+01]
  Bounds range     [1e+00, 9e+01]
  RHS range        [1e+00, 5e+01]
Presolve removed 12513 rows and 39063 columns
Presolve time: 0.31s
Presolved: 4023 rows, 39195 columns, 77757 nonzeros
Variable types: 0 continuous, 39195 integer (5320 binary)

Root relaxation: objective 6.812744e+03, 3849 iterations, 0.48 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6812.7438462 6812.74385  0.00%     -    1s

Explored 0 

Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16536 rows, 78258 columns and 152652 nonzeros
Model fingerprint: 0xde761cd6
Variable types: 0 continuous, 78258 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+02]
  Bounds range     [1e+00, 5e+01]
  RHS range        [1e+00, 8e+00]
Presolve removed 12614 rows and 58956 columns
Presolve time: 0.14s
Presolved: 3922 rows, 19302 columns, 38358 nonzeros
Variable types: 0 continuous, 19302 integer (5119 binary)

Root relaxation: objective 8.734846e+03, 300 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    8734.8461538 8734.84615  0.00%     -    0s

Explored 0 n

Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16536 rows, 78258 columns and 152652 nonzeros
Model fingerprint: 0xfa682ed8
Variable types: 0 continuous, 78258 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 2e+02]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 8e+01]
Presolve removed 12446 rows and 26443 columns
Presolve time: 0.19s
Presolved: 4090 rows, 51815 columns, 102101 nonzeros
Variable types: 0 continuous, 51815 integer (4203 binary)

Root relaxation: objective 5.966764e+04, 3583 iterations, 0.13 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    59667.644231 59667.6442  0.00%     -    0s

Explored 0

14×184×21×27 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.

In [20]:
sol_string = run_heur_route(disp_all_E, graphe, dims, usines, fournisseurs, emballages)
file_output = "france sol.txt"
open(joinpath("..", "sujet", file_output), "w") do file
    for ln in sol_string
        write(file, ln) 
        write(file, '\n') 
    end
end

routes = [lire_route(sol_string[1+r]) for r = 1:length(sol_string)-1]
instance_solved = lire_solution(instance, joinpath("..", "sujet", file_output))
@show feasibility(instance_solved)
totalcost = cost(instance_solved, verbose=true)
@show totalcost

feasibility(instance_solved) = true
Usine 1
   Jour 1
      Coût stock: 0
   Jour 2
      Coût stock: 0
   Jour 3
      Coût stock: 0
   Jour 4
      Coût stock: 0
   Jour 5
      Coût stock: 0
   Jour 6
      Coût stock: 0
   Jour 7
      Coût stock: 0
   Jour 8
      Coût stock: 0
   Jour 9
      Coût stock: 0
   Jour 10
      Coût stock: 0
   Jour 11
      Coût stock: 0
   Jour 12
      Coût stock: 0
   Jour 13
      Coût stock: 0
   Jour 14
      Coût stock: 0
   Jour 15
      Coût stock: 0
   Jour 16
      Coût stock: 0
   Jour 17
      Coût stock: 0
   Jour 18
      Coût stock: 0
   Jour 19
      Coût stock: 0
   Jour 20
      Coût stock: 0
   Jour 21
      Coût stock: 0
Usine 2
   Jour 1
      Coût stock: 0
   Jour 2
      Coût stock: 0
   Jour 3
      Coût stock: 0
   Jour 4
      Coût stock: 0
   Jour 5
      Coût stock: 0
   Jour 6
      Coût stock: 0
   Jour 7
      Coût stock: 0
   Jour 8
      Coût stock: 0
   Jour 9
      Coût stock: 0
   Jour 10
      Coût stock: 0
   Jo

      Coût expédition: 0
      Coût total: 0
   Jour 21
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
Fournisseur 2
   Jour 1
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 2
      Coût stock: 0
      Coût expédition: 600
      Coût total: 600
   Jour 3
      Coût stock: 0
      Coût expédition: 600
      Coût total: 600
   Jour 4
      Coût stock: 0
      Coût expédition: 600
      Coût total: 600
   Jour 5
      Coût stock: 0
      Coût expédition: 600
      Coût total: 600
   Jour 6
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 7
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 8
      Coût stock: 0
      Coût expédition: 300
      Coût total: 300
   Jour 9
      Coût stock: 0
      Coût expédition: 300
      Coût total: 300
   Jour 10
      Coût stock: 0
      Coût expédition: 510
      Coût total: 510
   Jour 11
      Coût stock: 0
      Coût expédition: 720
      Coût total: 720
   Jou

      Coût total: 0
   Jour 8
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 9
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 10
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 11
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 12
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 13
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 14
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 15
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 16
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 17
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 18
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 19
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 20
      Coût stock: 0
      Coût expédition: 0
      Coût tot

      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 13
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 14
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 15
      Coût stock: 0
      Coût expédition: 140
      Coût total: 140
   Jour 16
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 17
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 18
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 19
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 20
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 21
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
Fournisseur 15
   Jour 1
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 2
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 3
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour

   Jour 16
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 17
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 18
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 19
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 20
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 21
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
Fournisseur 21
   Jour 1
      Coût stock: 0
      Coût expédition: 100
      Coût total: 100
   Jour 2
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 3
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 4
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 5
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 6
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 7
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0


...

   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 200
   Nb camions: 1
   Coût total: 390
Route 1474, jour 13
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 200
   Nb camions: 1
   Coût total: 390
Route 1475, jour 14
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 651
   Nb camions: 1
   Coût total: 961
Route 1476, jour 14
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 255
   Nb camions: 1
   Coût total: 565
Route 1477, jour 14
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 311
   Nb camions: 1
   Coût total: 621
Route 1478, jour 14
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 1180
   Nb camions: 1
   Coût total: 1490
Route 1479, jour 14
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 1936
   Nb camions: 1
   Coût total: 2246
Route 1480, jour 14
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 587
   Nb camions: 1
   Coût total: 897
Route 1481, jour 14
   Coût camion: 150
  

   Coût total: 325
Route 1543, jour 17
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 216
   Nb camions: 1
   Coût total: 406
Route 1544, jour 17
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 273
   Nb camions: 1
   Coût total: 463
Route 1545, jour 17
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 70
   Nb camions: 1
   Coût total: 260
Route 1546, jour 17
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 195
   Nb camions: 1
   Coût total: 385
Route 1547, jour 17
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 4
   Nb camions: 1
   Coût total: 194
Route 1548, jour 17
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 216
   Nb camions: 1
   Coût total: 406
Route 1549, jour 17
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 52
   Nb camions: 1
   Coût total: 242
Route 1550, jour 17
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 34
   Nb camions: 1
   Coût total: 224
Route 1551, jour 1

Excessive output truncated after 524292 bytes.

Route 1591, jour 18
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 624
   Nb camions: 1
   Coût total: 814
Route 1592, jour 18
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 132
   Nb camions: 1
   Coût total: 322
Route 1593, jour 18
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 285
   Nb camions: 1
   Coût total: 475
Route 1594, jour 18
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 49
   Nb camions: 1
   Coût total: 239
Route 1595, jour 18
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 280
   Nb camions: 1
   Coût total: 470
Route 1596, jour 18
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 173
   Nb camions: 1
   Coût total: 363
Route 1597, jour 18
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 71
   Nb camions: 1
   Coût total: 261
Route 1598, jour 18
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 69
   Nb camions: 1
   Coût total: 259
Route 1599, jour 18

2090854

In [21]:
feasibility(instance_solved)

true

In [22]:
# france
totalcost

2090854

In [26]:
instance_file = "maroc.csv"
dims, emballages, usines,fournisseurs,graphe, instance = read_instance(instance_file)
disp_all_E = get_disp_all_E(graphe, dims, usines, fournisseurs, emballages)
sol_string = run_heur_route(disp_all_E, graphe, dims, usines, fournisseurs, emballages)
file_output = "maroc sol.txt"
open(joinpath("..", "sujet", file_output), "w") do file
    for ln in sol_string
        write(file, ln) 
        write(file, '\n') 
    end
end

routes = [lire_route(sol_string[1+r]) for r = 1:length(sol_string)-1]
instance_solved = lire_solution(instance, joinpath("..", "sujet", file_output))
@show feasibility(instance_solved)
totalcost = cost(instance_solved, verbose=true)
@show totalcost

-34.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1148 rows, 2170 columns and 4088 nonzeros
Model fingerprint: 0xcc0fc36b
Variable types: 0 continuous, 2170 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 1e+02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 883 rows and 1395 columns
Presolve time: 0.00s
Presolved: 265 rows, 775 columns, 1536 nonzeros
Variable types: 0 continuous, 775 integer (252 binary)

Root relaxation: objective 2.014846e+03, 723 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    2014.8461538 2014.84615  0.00%     -    0s

Explored 0 nodes (723

-78.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1148 rows, 2170 columns and 4088 nonzeros
Model fingerprint: 0xe372f622
Variable types: 0 continuous, 2170 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 6e+00]
Presolve removed 882 rows and 1371 columns
Presolve time: 0.00s
Presolved: 266 rows, 799 columns, 1561 nonzeros
Variable types: 0 continuous, 799 integer (236 binary)

Root relaxation: objective 1.852664e+03, 540 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1852.6640000 1852.66400  0.00%     -    0s

Explored 0 nodes (540

Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1148 rows, 2170 columns and 4088 nonzeros
Model fingerprint: 0x8136e0b2
Variable types: 0 continuous, 2170 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 21, time in user-callback 0.00 sec
0.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4

-34.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1148 rows, 2170 columns and 4088 nonzeros
Model fingerprint: 0x601c402d
Variable types: 0 continuous, 2170 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 5e+02]
  Bounds range     [1e+00, 7e+00]
  RHS range        [2e+00, 2e+00]
Presolve removed 883 rows and 1395 columns
Presolve time: 0.00s
Presolved: 265 rows, 775 columns, 1536 nonzeros
Variable types: 0 continuous, 775 integer (252 binary)

Root relaxation: objective 2.259623e+04, 743 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    22596.230769 22596.2308  0.00%     -    0s

Explored 0 nodes (743

      Coût total: 9765
Fournisseur 5
   Jour 1
      Coût stock: 0
      Coût expédition: 120
      Coût total: 120
   Jour 2
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 3
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 4
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 5
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 6
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 7
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 8
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 9
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 10
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 11
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 12
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 13
      Coût stock: 0
      Coût expédition: 0


      Coût expédition: 0
      Coût total: 0
   Jour 21
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
Fournisseur 11
   Jour 1
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 2
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 3
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 4
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 5
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 6
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 7
      Coût stock: 8
      Coût expédition: 0
      Coût total: 8
   Jour 8
      Coût stock: 8
      Coût expédition: 0
      Coût total: 8
   Jour 9
      Coût stock: 8
      Coût expédition: 0
      Coût total: 8
   Jour 10
      Coût stock: 225
      Coût expédition: 0
      Coût total: 225
   Jour 11
      Coût stock: 225
      Coût expédition: 0
      Coût total: 225
   Jour 12
      Coût stock: 

   Coût kilométrique: 5
   Nb camions: 1
   Coût total: 195
Route 43, jour 5
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 5
   Nb camions: 1
   Coût total: 195
Route 44, jour 5
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 25
   Nb camions: 1
   Coût total: 215
Route 45, jour 6
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 247
   Nb camions: 1
   Coût total: 557
Route 46, jour 6
   Coût camion: 150
   Coût arrêts: 80
   Coût kilométrique: 401
   Nb camions: 1
   Coût total: 631
Route 47, jour 6
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 25
   Nb camions: 1
   Coût total: 215
Route 48, jour 6
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 5
   Nb camions: 1
   Coût total: 195
Route 49, jour 6
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 5
   Nb camions: 1
   Coût total: 195
Route 50, jour 6
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 25
   Nb camions: 1
   Coût total: 215
Rout

368509

In [27]:
feasibility(instance_solved)

true

In [28]:
# maroc
totalcost

368509

In [23]:
instance_file = "espagne.csv"
dims, emballages, usines,fournisseurs,graphe, instance = read_instance(instance_file)
disp_all_E = get_disp_all_E(graphe, dims, usines, fournisseurs, emballages)
sol_string = run_heur_route(disp_all_E, graphe, dims, usines, fournisseurs, emballages)
file_output = "espagne sol.txt"
open(joinpath("..", "sujet", file_output), "w") do file
    for ln in sol_string
        write(file, ln) 
        write(file, '\n') 
    end
end

routes = [lire_route(sol_string[1+r]) for r = 1:length(sol_string)-1]
instance_solved = lire_solution(instance, joinpath("..", "sujet", file_output))
@show feasibility(instance_solved)
totalcost = cost(instance_solved, verbose=true)
@show totalcost

23.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9989 rows, 28861 columns and 55349 nonzeros
Model fingerprint: 0x1bd71ef0
Variable types: 0 continuous, 28861 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 1e+02]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 8e+00]
Presolve removed 7574 rows and 19264 columns
Presolve time: 0.05s
Presolved: 2415 rows, 9597 columns, 18941 nonzeros
Variable types: 0 continuous, 9597 integer (2266 binary)

Root relaxation: objective 7.299423e+03, 431 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7299.4230769 7299.42308  0.00%     -    0s

Explored 0 n

-168.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9989 rows, 28861 columns and 55349 nonzeros
Model fingerprint: 0x96edd9e4
Variable types: 0 continuous, 28861 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 5e+01]
  Bounds range     [1e+00, 3e+02]
  RHS range        [1e+00, 6e+01]
Presolve removed 7513 rows and 12074 columns
Presolve time: 0.08s
Presolved: 2476 rows, 16787 columns, 33361 nonzeros
Variable types: 0 continuous, 16787 integer (2424 binary)

Root relaxation: objective 5.551849e+03, 2795 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5551.8492308 5551.84923  0.00%     -    0s

Explore

0.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9989 rows, 28861 columns and 55349 nonzeros
Model fingerprint: 0xbbc0fd97
Variable types: 0 continuous, 28861 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

User-callback calls 21, time in user-callback 0.00 sec
-6.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread 

Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9989 rows, 28861 columns and 55349 nonzeros
Model fingerprint: 0xd634f203
Variable types: 0 continuous, 28861 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e+00, 2e+02]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 2e+01]
Presolve removed 7560 rows and 16837 columns
Presolve time: 0.22s
Presolved: 2429 rows, 12024 columns, 23767 nonzeros
Variable types: 0 continuous, 12024 integer (3398 binary)

Root relaxation: objective 1.207909e+04, 676 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    12079.086538 12079.0865  0.00%     -    0s

Explored 0 node

      Coût stock: 1744
   Jour 21
      Coût stock: 1744
Usine 6
   Jour 1
      Coût stock: 0
   Jour 2
      Coût stock: 0
   Jour 3
      Coût stock: 42
   Jour 4
      Coût stock: 42
   Jour 5
      Coût stock: 84
   Jour 6
      Coût stock: 0
   Jour 7
      Coût stock: 0
   Jour 8
      Coût stock: 0
   Jour 9
      Coût stock: 0
   Jour 10
      Coût stock: 0
   Jour 11
      Coût stock: 0
   Jour 12
      Coût stock: 0
   Jour 13
      Coût stock: 0
   Jour 14
      Coût stock: 0
   Jour 15
      Coût stock: 3
   Jour 16
      Coût stock: 6
   Jour 17
      Coût stock: 121
   Jour 18
      Coût stock: 2023
   Jour 19
      Coût stock: 3990
   Jour 20
      Coût stock: 3990
   Jour 21
      Coût stock: 3990

Fournisseur 1
   Jour 1
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 2
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 3
      Coût stock: 0
      Coût expédition: 0
      Coût total: 0
   Jour 4
      Coût stock: 0
      

...

   Coût total: 662
Route 1349, jour 9
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 431
   Nb camions: 1
   Coût total: 621
Route 1350, jour 9
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 835
   Nb camions: 1
   Coût total: 1025
Route 1351, jour 9
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 680
   Nb camions: 1
   Coût total: 870
Route 1352, jour 10
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 539
   Nb camions: 1
   Coût total: 849
Route 1353, jour 10
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 627
   Nb camions: 1
   Coût total: 937
Route 1354, jour 10
   Coût camion: 150
   Coût arrêts: 120
   Coût kilométrique: 714
   Nb camions: 1
   Coût total: 984
Route 1355, jour 10
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 451
   Nb camions: 1
   Coût total: 641
Route 1356, jour 10
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 696
   Nb camions: 1
   Coût total: 886
Route 1357, 

   Coût total: 830
Route 1437, jour 7
   Coût camion: 150
   Coût arrêts: 80
   Coût kilométrique: 245
   Nb camions: 1
   Coût total: 475
Route 1438, jour 8
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 2054
   Nb camions: 1
   Coût total: 2364
Route 1439, jour 8
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 639
   Nb camions: 1
   Coût total: 829
Route 1440, jour 8
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 2
   Nb camions: 1
   Coût total: 192
Route 1441, jour 8
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 392
   Nb camions: 1
   Coût total: 582
Route 1442, jour 9
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 605
   Nb camions: 1
   Coût total: 915
Route 1443, jour 9
   Coût camion: 150
   Coût arrêts: 120
   Coût kilométrique: 1751
   Nb camions: 1
   Coût total: 2021
Route 1444, jour 9
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 293
   Nb camions: 1
   Coût total: 483
Route 1445, jour

Route 1514, jour 3
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 33
   Nb camions: 1
   Coût total: 223
Route 1515, jour 3
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 394
   Nb camions: 1
   Coût total: 584
Route 1516, jour 3
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 204
   Nb camions: 1
   Coût total: 394
Route 1517, jour 3
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 640
   Nb camions: 1
   Coût total: 830
Route 1518, jour 3
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 639
   Nb camions: 1
   Coût total: 829
Route 1519, jour 3
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 71
   Nb camions: 1
   Coût total: 261
Route 1520, jour 3
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 408
   Nb camions: 1
   Coût total: 598
Route 1521, jour 3
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 230
   Nb camions: 1
   Coût total: 420
Route 1522, jour 3
   Coût camion: 150
   

   Coût kilométrique: 392
   Nb camions: 1
   Coût total: 582
Route 1585, jour 9
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 246
   Nb camions: 1
   Coût total: 436
Route 1586, jour 9
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 358
   Nb camions: 1
   Coût total: 548
Route 1587, jour 10
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 428
   Nb camions: 1
   Coût total: 738
Route 1588, jour 10
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 545
   Nb camions: 1
   Coût total: 855
Route 1589, jour 10
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 456
   Nb camions: 1
   Coût total: 766
Route 1590, jour 10
   Coût camion: 150
   Coût arrêts: 160
   Coût kilométrique: 1334
   Nb camions: 1
   Coût total: 1644
Route 1591, jour 10
   Coût camion: 150
   Coût arrêts: 120
   Coût kilométrique: 1324
   Nb camions: 1
   Coût total: 1594
Route 1592, jour 10
   Coût camion: 150
   Coût arrêts: 40
   Coût kilométrique: 394

1322729

In [24]:
feasibility(instance_solved)

true

In [25]:
totalcost

1322729

In [12]:
instance_file = "petite.csv"
dims, emballages, usines,fournisseurs,graphe, instance = read_instance(instance_file)
disp_all_E = get_disp_all_E(graphe, dims, usines, fournisseurs, emballages)
sol_string = run_heur_route(disp_all_E, graphe, dims, usines, fournisseurs, emballages)
file_output = "petite sol.txt"
open(joinpath("..", "sujet", file_output), "w") do file
    for ln in sol_string
        write(file, ln) 
        write(file, '\n') 
    end
end

routes = [lire_route(sol_string[1+r]) for r = 1:length(sol_string)-1]
instance_solved = lire_solution(instance, joinpath("..", "sujet", file_output))
@show feasibility(instance_solved)
totalcost = cost(instance_solved, verbose=true)
@show totalcost

-26.0
Academic license - for non-commercial use only - expires 2021-02-15
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 59 rows, 91 columns and 173 nonzeros
Model fingerprint: 0x2678c77a
Variable types: 0 continuous, 91 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+01]
  Bounds range     [3e+00, 6e+00]
  RHS range        [2e+00, 1e+01]
Found heuristic solution: objective 468.4615385
Presolve removed 44 rows and 51 columns
Presolve time: 0.00s
Presolved: 15 rows, 40 columns, 71 nonzeros
Found heuristic solution: objective 450.7307692
Variable types: 0 continuous, 40 integer (0 binary)

Root relaxation: objective 4.776923e+01, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0       

730

In [13]:
feasibility(instance_solved)

true

In [14]:
totalcost

730